# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `production_2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [72]:
# Write your code below.
from dotenv import load_dotenv
import pandas as pd
import os
import sys
env_path = r"C:\Users\JEFFEY MARKUS\production\05_src\.env"
load_dotenv(dotenv_path=env_path)
price_data_path = os.getenv('PRICE_DATA')
print(price_data_path)

../05_src/data/prices/


+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [60]:
from glob import glob
import dask.dataframe as dd

price_data_path = r'C:\Users\JEFFEY MARKUS\production\05_src\data\prices'
parquet_files = []

for root, dirs, files in os.walk(price_data_path):
    for file in files:
        if file.endswith('.parquet'):
            parquet_files.append(os.path.join(root, file))

df = dd.read_parquet(parquet_files)
print(df.head())

Price                       Date  Adj Close      Close       High        Low  \
ticker                                                                         
A      2000-01-03 00:00:00+00:00  43.463024  51.502148  56.464592  48.193848   
A      2000-01-04 00:00:00+00:00  40.142933  47.567955  49.266811  46.316166   
A      2000-01-05 00:00:00+00:00  37.652870  44.617310  47.567955  43.141991   
A      2000-01-06 00:00:00+00:00  36.219189  42.918453  44.349072  41.577251   
A      2000-01-07 00:00:00+00:00  39.237465  46.494991  47.165592  42.203148   

Price        Open   Volume       sector                       subsector  year  
ticker                                                                         
A       56.330471  4674353  Health Care  Life Sciences Tools & Services  2000  
A       48.730328  4765083  Health Care  Life Sciences Tools & Services  2000  
A       47.389126  5758642  Health Care  Life Sciences Tools & Services  2000  
A       44.080830  2534434  Health Care

For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Adjusted Close:
    
    - `returns`: (Adj Close / Adj Close_lag) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [71]:
#checking the names of the columns
print(df.columns)

Index(['Date', 'Adj Close', 'Close', 'High', 'Low', 'Open', 'Volume', 'sector',
       'subsector', 'year', 'Close_lag', 'Adj_Close_lag', 'returns',
       'hi_lo_range'],
      dtype='object', name='Price')


In [70]:
# Added lag for 'Close' and 'Adj_Close'
df['Close_lag'] = df['Close'].shift(1)
df['Adj_Close_lag'] = df['Adj Close'].shift(1)
# Calculated returns: (Adj Close / Adj Close_lag) - 1
df['returns'] = (df['Adj Close'] / df['Adj_Close_lag']) - 1
# Add hi_lo_range: High minus Low
df['hi_lo_range'] = df['High'] - df['Low']
#Assigning results to ddfeat
ddfeat=df
ddfeat.head(10)


Price,Date,Adj Close,Close,High,Low,Open,Volume,sector,subsector,year,Close_lag,Adj_Close_lag,returns,hi_lo_range
ticker,,,,,,,,,,,,,,
A,2000-01-03 00:00:00+00:00,43.463024,51.502148,56.464592,48.193848,56.330471,4674353,Health Care,Life Sciences Tools & Services,2000,NaN,NaN,NaN,8.270744
A,2000-01-04 00:00:00+00:00,40.142933,47.567955,49.266811,46.316166,48.730328,4765083,Health Care,Life Sciences Tools & Services,2000,51.502148,43.463024,-0.076389,2.950645
A,2000-01-05 00:00:00+00:00,37.652870,44.617310,47.567955,43.141991,47.389126,5758642,Health Care,Life Sciences Tools & Services,2000,47.567955,40.142933,-0.062030,4.425964
A,2000-01-06 00:00:00+00:00,36.219189,42.918453,44.349072,41.577251,44.080830,2534434,Health Care,Life Sciences Tools & Services,2000,44.617310,37.652870,-0.038076,2.771820
A,2000-01-07 00:00:00+00:00,39.237465,46.494991,47.165592,42.203148,42.247852,2819626,Health Care,Life Sciences Tools & Services,2000,42.918453,36.219189,0.083334,4.962444
A,2000-01-10 00:00:00+00:00,41.614338,49.311516,49.803291,48.327969,49.356224,2148446,Health Care,Life Sciences Tools & Services,2000,46.494991,39.237465,0.060577,1.475323
A,2000-01-11 00:00:00+00:00,41.048424,48.640915,49.311516,47.523247,49.311516,1855985,Health Care,Life Sciences Tools & Services,2000,49.311516,41.614338,-0.013599,1.788269
A,2000-01-12 00:00:00+00:00,40.218384,47.657368,48.640915,45.824390,48.640915,1429874,Health Care,Life Sciences Tools & Services,2000,48.640915,41.048424,-0.020221,2.816525
A,2000-01-13 00:00:00+00:00,40.822041,48.372677,49.937412,47.210300,48.909157,1134337,Health Care,Life Sciences Tools & Services,2000,47.657368,40.218384,0.015009,2.727112


+ Convert the Dask data frame to a pandas data frame. 
+ Add a rolling average return calculation with a window of 10 days.
+ *Tip*: Consider using `.rolling(10).mean()`.

(3 pt)

In [75]:
# Write your code below.

# Converting the Dask DataFrame to a Pandas DataFrame
df_pandas = ddfeat.compute()

# Adding a rolling average return calculation with a 10-day window
df_pandas['rolling_avg_return'] = df_pandas['returns'].rolling(window=10).mean()

# Displaying the first few rows to verify the rolling average return calculation
print(df_pandas[['returns', 'rolling_avg_return']].head(15)) 


Price    returns  rolling_avg_return
ticker                              
A            NaN                 NaN
A      -0.076389                 NaN
A      -0.062030                 NaN
A      -0.038076                 NaN
A       0.083334                 NaN
A       0.060577                 NaN
A      -0.013599                 NaN
A      -0.020221                 NaN
A       0.015009                 NaN
A       0.011091                 NaN
A       0.045704            0.000540
A      -0.020979            0.006081
A      -0.026786            0.009605
A       0.009175            0.014330
A      -0.003637            0.005633


Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?
+ Would it have been better to do it in Dask? Why?

(1 pt)

In [95]:
print("1. It is not necessary to convert the Dask dataframe into a Pandas to calculate the moving average return."
      "\n   It could have been done with Dask itself as it supports rolling window mean calculations.\n")
print("2. YES!!! It would be significantly better to do it in Dask." 
      "\n   Unlike Pandas, which loads the entire dataframe into memory," 
      "\n   Dask is lazy, memory efficient and only evaluates when necessary."
      "\n   making it suitable for handling large scale datasets")


1. It is not necessary to convert the Dask dataframe into a Pandas to calculate the moving average return.
   It could have been done with Dask itself as it supports rolling window mean calculations.

2. YES!!! It would be significantly better to do it in Dask.
   Unlike Pandas, which loads the entire dataframe into memory,
   Dask is lazy, memory efficient and only evaluates when necessary.
   making it suitable for handling large scale datasets


## Criteria

The [rubric](./assignment_1_rubric_clean.xlsx) contains the criteria for grading.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.